In [1]:
import tkinter
import tkinter.messagebox
import customtkinter
import tkinter.messagebox as messagebox
from tkintermapview import TkinterMapView
from tkinter import filedialog as fd
from pathlib import Path
from PIL import Image

from getlocationtype import *
from getlocationflooddata import *

customtkinter.set_appearance_mode("Dark")  # Modes: "System" (standard), "Dark", "Light"
customtkinter.set_default_color_theme("dark-blue")  # Themes: "blue" (standard), "green", "dark-blue"

class App(customtkinter.CTk):
    def __init__(self):
        self.lon = 0
        self.lat = 0

        super().__init__()

        # change app icon
        self.iconbitmap("logo.ico")

        # configure window
        self.title("Project T")
        self.geometry(f"{1280}x{780}")

        # configure grid layout (4x4)
        self.grid_columnconfigure((1, 2), weight=1)
        self.grid_rowconfigure((0, 1), weight=1)

        # create sidebar frame with widgets
        self.sidebar_frame = customtkinter.CTkFrame(self, fg_color=("white", "#181a29"), corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=7, sticky="nsw")
        self.sidebar_frame.grid_rowconfigure(8, weight=1)

        # button appearances & paddings
        x1 = 20
        x2 = 20
        y1 = 10
        y2 = 0
        navbarcolors = ("#8870e6", "#8870e6")
        navbarhover = ("#4d4c71","#4d4c71")
        # bgcolorframe = ("white", "#181a29")
        bgcolorframe = ("red", "blue")
        buttoncolors = ("#8870e6","#8870e6")

        # bg color
        self.background_frame = customtkinter.CTkFrame(self, fg_color=("#f4f5fe", "#0e0f14"), corner_radius=0, height=2000, width=2000)
        self.background_frame.grid(row=0, column=0, rowspan=7, columnspan=4, sticky="nsew")
        self.background_frame.grid_rowconfigure(7, weight=1)
        self.background_frame.grid_columnconfigure(3, weight=1)
        self.sidebar_frame.lift() # move the background frame to the back
        self.background_frame.lower()  # move the background frame to the bottom

        # Title 
        self.app_title = customtkinter.CTkLabel(self.sidebar_frame, text="Project T", font=customtkinter.CTkFont(size=12, weight="bold"))
        self.app_title.grid(row=0, column=0, columnspan=2, padx=20, pady=(20, 10))

        # Banner
        my_image = customtkinter.CTkImage(light_image=Image.open("TITLE.png"),
                            dark_image=Image.open("TITLE.png"),
                            size=(30, 30))
        self.app_banner = customtkinter.CTkLabel(self.sidebar_frame, text="", image=my_image)
        self.app_banner.grid(row=1, column=0, columnspan=2, padx=20, pady=(0, 0))

       # Navigation bar through buttons
        self.button1 = customtkinter.CTkButton(self.sidebar_frame, text="Location & Model", fg_color="transparent", text_color= navbarcolors, hover_color=navbarhover, corner_radius=0, command=self.show_option1, anchor="w", font=customtkinter.CTkFont(size=12, weight="bold"))
        self.button1.grid(row=3, column=0, padx=(x1, x2), pady=(50, y2), sticky="we", columnspan=2)

        self.button2 = customtkinter.CTkButton(self.sidebar_frame, text="Flood Impact", fg_color="transparent", text_color= navbarcolors, hover_color=navbarhover, corner_radius=0, command=self.show_option2, anchor="w", font=customtkinter.CTkFont(size=12, weight="bold"))
        self.button2.grid(row=4, column=0, padx=(x1, x2), pady=(y1, y2), sticky="we", columnspan=2)

        self.button3 = customtkinter.CTkButton(self.sidebar_frame, text="Temeperature Impact", fg_color="transparent", text_color= navbarcolors, hover_color=navbarhover, corner_radius=0, command=self.show_option3, anchor="w", font=customtkinter.CTkFont(size=12, weight="bold"))
        self.button3.grid(row=5, column=0, padx=(x1, x2), pady=(y1, y2), sticky="we", columnspan=2)

        self.button4 = customtkinter.CTkButton(self.sidebar_frame, text="Resilience Results", fg_color="transparent", text_color= navbarcolors, hover_color=navbarhover, corner_radius=0, command=self.show_option4, anchor="w", font=customtkinter.CTkFont(size=12, weight="bold"))
        self.button4.grid(row=6, column=0, padx=(x1, x2), pady=(y1, y2), sticky="we", columnspan=2)

        # UI settings
        self.appearance_mode_label = customtkinter.CTkLabel(self.sidebar_frame, text="Theme:", anchor="w")
        self.appearance_mode_label.grid(row=9, column=0, padx=(20,20), pady=(0, 10), sticky = "s")
        self.appearance_mode_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["Light", "Dark"],
                                                                       command=self.change_appearance_mode_event, dropdown_text_color= "white", fg_color=buttoncolors, button_color=buttoncolors, button_hover_color=("#4d4c71","#4d4c71"), dropdown_fg_color= ("#4d4c71","#4d4c71"), dropdown_hover_color=("#4d4c71","#4d4c71"))
        self.appearance_mode_optionemenu.grid(row=9, column=1, padx=(20,20), pady=(10, 10), sticky = "s")
        self.scaling_label = customtkinter.CTkLabel(self.sidebar_frame, text="UI Scaling:", anchor="w")
        self.scaling_label.grid(row=10, column=0, padx=(20,20), pady=(0, 50), sticky = "s")
        self.scaling_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["100%", "110%", "120%", "150%"],
                                                               command=self.change_scaling_event, dropdown_text_color= "white", fg_color=buttoncolors, button_color=buttoncolors, button_hover_color=("#4d4c71","#4d4c71"), dropdown_fg_color= ("#4d4c71","#4d4c71"), dropdown_hover_color=("#4d4c71","#4d4c71"))
        self.scaling_optionemenu.grid(row=10, column=1, padx=(20,20), pady=(0, 50), sticky = "s")

        # create scrollable frame for displaying options
        # self.scrollable_frame = customtkinter.CTkScrollableFrame(self, fg_color = ("white", "#181a29"), bg_color = ("white", "#181a29"), scrollbar_button_color=("white", "#181a29"), scrollbar_button_hover_color=("white", "#181a29"))
        self.scrollable_frame = customtkinter.CTkFrame(self, fg_color = bgcolorframe, bg_color = bgcolorframe)
        self.scrollable_frame.grid(row=0, column=1, rowspan=7, columnspan=5, padx=(30, 30), pady=(30, 30), sticky="nsew")
        # self.scrollable_frame.grid_rowconfigure(0, weight=1)

        # create content for each option
        # setup tab
        self.option_content = customtkinter.CTkLabel(self.scrollable_frame, text="WELCOME", fg_color = bgcolorframe, bg_color = bgcolorframe)
        self.option_content.grid(row=0, column=0, padx=20, pady=20, sticky="w")

        self.option1_content = customtkinter.CTkLabel(self.scrollable_frame, text="Location & Model Configuration", bg_color = bgcolorframe, fg_color = bgcolorframe)
        self.option1_content.grid(row=0, column=0, padx=20, pady=20, sticky="w")

        self.loadifc_button = customtkinter.CTkButton(self.option1_content, fg_color=buttoncolors, bg_color = bgcolorframe, hover_color=("#4d4c71","#4d4c71"), text="Load IFC file",command=self.ifcload, font=customtkinter.CTkFont(size=12, weight="bold"))
        self.loadifc_button.grid(row=2, column=0, padx=20, pady=10, sticky="w")

        self.entry = customtkinter.CTkEntry(master=self.option1_content, bg_color = bgcolorframe,fg_color = bgcolorframe, placeholder_text="type address", width=150)
        self.entry.grid(row=3, column=0, sticky="we", padx=(30, 0), pady=12)
        self.entry.bind("<Return>", self.search_event)

        self.searchbutton = customtkinter.CTkButton(master=self.option1_content, text="Search", bg_color = bgcolorframe, fg_color=buttoncolors, hover_color=("#4d4c71","#4d4c71"), width=30, command=self.search_event,font=customtkinter.CTkFont(size=12, weight="bold"))
        self.searchbutton.grid(row=1, column=2, sticky="we", padx=(30, 30), pady=12)

        self.map_widget = TkinterMapView(self.option1_content, corner_radius=0, height=360, bg_color=bgcolorframe)
        self.map_widget.grid(row=1, column=1, sticky="nswe", padx=(20, 20), pady=(0, 0))
        self.map_widget.add_right_click_menu_command(label="Select location", command=self.add_marker_event,pass_coords=True)

        self.option2_content = customtkinter.CTkLabel(self.scrollable_frame, text="Option 2 content")
        self.option2_content.grid(row=0, column=0, padx=20, pady=20, sticky="nsew")

        self.entry = customtkinter.CTkEntry(master=self.option2_content,
                                            placeholder_text="type address", width=150)
        self.entry.grid(row=3, column=0, sticky="we", padx=(30, 0), pady=12)
        self.entry.bind("<Return>", self.search_event)

        self.searchbutton = customtkinter.CTkButton(master=self.option2_content, text="Search", fg_color=("#8870e6","#8870e6"), hover_color=("#4d4c71","#4d4c71"), width=30, command=self.search_event,font=customtkinter.CTkFont(size=12, weight="bold"))
        self.searchbutton.grid(row=3, column=1, sticky="we", padx=(30, 30), pady=12)

        self.option3_content = customtkinter.CTkLabel(self.scrollable_frame, text="Option 3 content")
        self.option3_content.grid(row=0, column=0, padx=20, pady=20, sticky="nsew")

        self.option4_content = customtkinter.CTkLabel(self.scrollable_frame, text="Option 4 content")
        self.option4_content.grid(row=0, column=0, padx=20, pady=20, sticky="nsew")

        # set default values
        self.appearance_mode_optionemenu.set("Dark")
        self.scaling_optionemenu.set("100%")

        # hide all option contents initially
        self.option1_content.grid_remove()
        self.option2_content.grid_remove()
        self.option3_content.grid_remove()
        self.option4_content.grid_remove()

    def show_option1(self):
        # hide other option contents and show option 1 content
        self.option2_content.grid_remove()
        self.option3_content.grid_remove()
        self.option4_content.grid_remove()
        self.option1_content.grid()
    
    def show_option2(self):
        # hide other option contents and show option 2 content
        self.option1_content.grid_remove()
        self.option3_content.grid_remove()
        self.option4_content.grid_remove()
        self.option2_content.grid()

    def show_option3(self):
        # hide other option contents and show option 3 content
        self.option1_content.grid_remove()
        self.option2_content.grid_remove()
        self.option4_content.grid_remove()
        self.option3_content.grid()

    def show_option4(self):
        # hide other option contents and show option 4 content
        self.option1_content.grid_remove()
        self.option2_content.grid_remove()
        self.option3_content.grid_remove()
        self.option4_content.grid()

    # Change theme and UI scale
    def change_appearance_mode_event(self, new_appearance_mode: str):
        customtkinter.set_appearance_mode(new_appearance_mode)

    def change_scaling_event(self, new_scaling: str):
        new_scaling_float = int(new_scaling.replace("%", "")) / 100
        customtkinter.set_widget_scaling(new_scaling_float)

    def ifcload(self):
            print("sidebar_button click")
            global filename

            filetypes = (('IFC files', '*.ifc'),('All files', '*.*'))

            filename = fd.askopenfilename(title='Open a file', initialdir="C:\\", filetypes=filetypes)
            filename = Path(filename).stem

            # Check if file extension is ".ifc"
            if Path(filename).suffix != ".ifc":
                messagebox.showerror("Error", "Invalid file type. Only IFC files are allowed.")
                return
            
            loadedifc = filename + ".ifc"
            print(loadedifc)
        
    def search_event(self, event=None):
        self.map_widget.set_address(self.entry.get())

    def add_marker_event(self, coords):
            self.map_widget.delete_all_marker()
            self.map_widget.delete_all_path()
            # self.map_widget.set_address(self.Location_adres.get(), marker=True)
            global lon
            lon = coords[0]
            global lat
            lat = coords[1]
            # # print(coords)
            # # print(self.lat, self.lon)
            # locationtype = gettypelocation(self.lon, self.lat)
            # print(coords)
            # print(self.lat, self.lon)
            locationtype = gettypelocation(lon, lat)
            print(locationtype, lon, lat)

    

if __name__ == "__main__":
    app = App()
    app.mainloop()